In [17]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import os
from argparse import ArgumentParser
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import TensorBoard

In [18]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output


class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.mlp = tf.keras.Sequential(
            [
                Dense(mlp_dim, activation=tf.nn.relu),
                Dense(embed_dim),
            ]
        )

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        out1 = attn_output + inputs

        mlp_output = self.mlp(out1)
        return mlp_output + out1


class VisionTransformer(tf.keras.Model):
    def __init__(self, image_size, patch_size, num_layers, num_classes, d_model, num_heads, mlp_dim, channels=1):
        super(VisionTransformer, self).__init__()
        num_patches = (image_size // patch_size) ** 2
        self.patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size
        self.d_model = d_model
        self.num_layers = num_layers

        self.rescale = Rescaling(1.0 / 255)
        self.pos_emb = self.add_weight(
            "pos_emb", shape=(1, num_patches + 1, d_model)
        )
        self.class_emb = self.add_weight("class_emb", shape=(1, 1, d_model))
        self.patch_proj = Dense(d_model)
        self.enc_layers = [
            TransformerBlock(d_model, num_heads, mlp_dim)
            for _ in range(num_layers)
        ]
        self.mlp_head = tf.keras.Sequential(
            [
                Dense(mlp_dim, activation=tfa.activations.gelu),
                Dense(num_classes),
            ]
        )

    def extract_patches(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patches = tf.reshape(patches, [batch_size, -1, self.patch_dim])
        return patches

    def call(self, x, training):
        batch_size = tf.shape(x)[0]
        x = self.rescale(x)
        patches = self.extract_patches(x)
        x = self.patch_proj(patches)

        class_emb = tf.broadcast_to(self.class_emb, [batch_size, 1, self.d_model])
        x = tf.concat([class_emb, x], axis=1)
        x = x + self.pos_emb

        for layer in self.enc_layers:
            x = layer(x, training)

        # First (class token) is used for classification
        x = self.mlp_head(x[:, 0])
        return x
    
    def load_weight(self, filepath):
        """Loads all layer weights, either from a TensorFlow or an HDF5 weight file.
        Arguments:
            filepath: String, path to the weights file to load. For weight files in
                TensorFlow format, this is the file prefix (the same as was passed
                to `save_weights`).
        Returns:
            When loading a weight file in TensorFlow format, returns the same status
            object as `tf.train.Checkpoint.restore`. When graph building, restore
            ops are run automatically as soon as the network is built (on first call
            for user-defined classes inheriting from `Model`, immediately if it is
            already built).
            When loading weights in HDF5 format, returns `None`.
        Raises:
            ImportError: If h5py is not available and the weight file is in HDF5
                format.
            ValueError: If `skip_mismatch` is set to `True` when `by_name` is
              `False`.
        """

        filepath = path_to_string(filepath)
        if h5py is None:
            raise ImportError(
                '`load_weights` requires h5py when loading weights from HDF5.')
        if not self._is_graph_network and not self.built:
            raise ValueError(
                'Unable to load weights saved in HDF5 format into a subclassed '
                'Model which has not created its variables yet. Call the Model '
                'first, then load the weights.')
        with h5py.File(filepath, 'r') as f:
            if 'layer_names' not in f.attrs and 'model_weights' in f:
                f = f['model_weights']
            else:
                hdf5_format.load_weights_from_hdf5_group(f, self.layers)

In [19]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

logdir = "logs"
image_size = 28
patch_size = 4
num_layers = 4
d_model = 64
num_heads = 4
mlp_dim = 128
lr = 3e-4
weight_decay = 1e-4
batch_size = 4096
epochs = 1

ds = tfds.load("mnist", as_supervised=True)

ds_train = (ds["train"].shuffle(5 * batch_size).batch(batch_size))

ds_test = (
    ds["test"]
    .cache()
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)


model = VisionTransformer(
    image_size=image_size,
    patch_size=patch_size,
    num_layers=num_layers,
    num_classes=10,
    d_model=d_model,
    num_heads=num_heads,
    mlp_dim=mlp_dim,
    channels=1,
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay),
    metrics=["accuracy"],
)
model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=epochs,
    callbacks=[TensorBoard(log_dir=logdir, profile_batch=0),],
)
model.load_weights("../saved_weights/vit.hdf5")
model.evaluate(ds_test)

3/3 [==============================] - 3s 982ms/step - loss: 3.9850 - accuracy: 0.1661


[3.9850449562072754, 0.16609999537467957]

In [20]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [23]:
model.summary()

Model: "vision_transformer_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_11 (Rescaling)     multiple                  0         
_________________________________________________________________
dense_297 (Dense)            multiple                  1088      
_________________________________________________________________
transformer_block_44 (Transf multiple                  33216     
_________________________________________________________________
transformer_block_45 (Transf multiple                  33216     
_________________________________________________________________
transformer_block_46 (Transf multiple                  33216     
_________________________________________________________________
transformer_block_47 (Transf multiple                  33216     
_________________________________________________________________
sequential_59 (Sequential)   (None, 10)      

In [41]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

logdir = "logs"
image_size = 28
patch_size = 4
num_layers = 4
d_model = 64
num_heads = 4
mlp_dim = 128
lr = 3e-4
weight_decay = 1e-4
batch_size = 4096
epochs = 300

ds = tfds.load("mnist", as_supervised=True)

ds_train = (
    ds["train"]
    .cache()
    .shuffle(5 * batch_size)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)
ds_test = (
    ds["test"]
    .cache()
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = VisionTransformer(
        image_size=image_size,
        patch_size=patch_size,
        num_layers=num_layers,
        num_classes=10,
        d_model=d_model,
        num_heads=num_heads,
        mlp_dim=mlp_dim,
        channels=1,
    )
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay),
        metrics=["accuracy"],
    )

model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=epochs,
    callbacks=[TensorBoard(log_dir=logdir, profile_batch=0),],
)
model.save_weights(os.path.join(logdir, "weights", "vit"))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Epoch 1/300
 2/15 [===>..........................] - ETA: 2s - loss: 2.3370 - accuracy: 0.1193WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1050s vs `on_train_batch_end` time: 0.1593s). Check your callbacks.


15/15 [==============================] - ETA: 0s - loss: 2.1984 - accuracy: 0.1835WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7fa7a8620430> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


15/15 [==============================] - 6s 382ms/step - loss: 2.1984 - accuracy: 0.1835 - val_loss: 2.0695 - val_accuracy: 0.2563
Epoch 2/300
15/15 [==============================] - 4s 275ms/step - loss: 1.9991 - accuracy: 0.2813 - val_loss: 1.9038 - val_accuracy: 0.3208
Epoch 3/300
15/15 [==============================] - 4s 268ms/step - loss: 1.7814 - accuracy: 0.3652 - val_loss: 1.5808 - val_accuracy: 0.4382
Epoch 4/300
15/15 [==============================] - 4s 269ms/step - loss: 1.4334 - accuracy: 0.4929 - val_loss: 1.2357 - val_accuracy: 0.5564
Epoch 5/300
15/15 [==============================] - 4s 270ms/step - loss: 1.1345 - accuracy: 0.6058 - val_loss: 0.9781 - val_accuracy: 0.6675
Epoch 6/300
15/15 [==============================] - 4s 270ms/step - loss: 0.8986 - accuracy: 0.6990 - val_loss: 0.7989 - val_accuracy: 0.7384
Epoch 7/300
15/15 [==============================] - 4s 269ms/step - loss: 0.7386 - accuracy: 0.7558 - val_loss: 0.6419 - val_accuracy: 0.7886
Epoch 8/300

15/15 [==============================] - 4s 268ms/step - loss: 0.1289 - accuracy: 0.9605 - val_loss: 0.1512 - val_accuracy: 0.9530
Epoch 59/300
15/15 [==============================] - 4s 269ms/step - loss: 0.1312 - accuracy: 0.9587 - val_loss: 0.1491 - val_accuracy: 0.9526
Epoch 60/300
15/15 [==============================] - 4s 269ms/step - loss: 0.1278 - accuracy: 0.9604 - val_loss: 0.1417 - val_accuracy: 0.9542
Epoch 61/300
15/15 [==============================] - 4s 268ms/step - loss: 0.1246 - accuracy: 0.9613 - val_loss: 0.1426 - val_accuracy: 0.9537
Epoch 62/300
15/15 [==============================] - 4s 269ms/step - loss: 0.1226 - accuracy: 0.9621 - val_loss: 0.1369 - val_accuracy: 0.9553
Epoch 63/300
15/15 [==============================] - 4s 269ms/step - loss: 0.1194 - accuracy: 0.9631 - val_loss: 0.1407 - val_accuracy: 0.9542
Epoch 64/300
15/15 [==============================] - 4s 268ms/step - loss: 0.1257 - accuracy: 0.9604 - val_loss: 0.1390 - val_accuracy: 0.9558
Epoch

Epoch 115/300
15/15 [==============================] - 4s 269ms/step - loss: 0.0732 - accuracy: 0.9778 - val_loss: 0.1099 - val_accuracy: 0.9671
Epoch 116/300
15/15 [==============================] - 4s 268ms/step - loss: 0.0721 - accuracy: 0.9780 - val_loss: 0.1074 - val_accuracy: 0.9668
Epoch 117/300
15/15 [==============================] - 4s 269ms/step - loss: 0.0715 - accuracy: 0.9781 - val_loss: 0.1074 - val_accuracy: 0.9667
Epoch 118/300
15/15 [==============================] - 4s 268ms/step - loss: 0.0745 - accuracy: 0.9769 - val_loss: 0.1067 - val_accuracy: 0.9669
Epoch 119/300
15/15 [==============================] - 4s 268ms/step - loss: 0.0733 - accuracy: 0.9766 - val_loss: 0.1047 - val_accuracy: 0.9674
Epoch 120/300
15/15 [==============================] - 4s 269ms/step - loss: 0.0703 - accuracy: 0.9785 - val_loss: 0.1046 - val_accuracy: 0.9680
Epoch 121/300
15/15 [==============================] - 4s 268ms/step - loss: 0.0696 - accuracy: 0.9789 - val_loss: 0.1136 - val_ac

15/15 [==============================] - 4s 267ms/step - loss: 0.0385 - accuracy: 0.9881 - val_loss: 0.0918 - val_accuracy: 0.9735
Epoch 228/300
15/15 [==============================] - 4s 268ms/step - loss: 0.0378 - accuracy: 0.9884 - val_loss: 0.0940 - val_accuracy: 0.9732
Epoch 229/300
15/15 [==============================] - 4s 267ms/step - loss: 0.0347 - accuracy: 0.9896 - val_loss: 0.0889 - val_accuracy: 0.9759
Epoch 230/300
15/15 [==============================] - 4s 266ms/step - loss: 0.0355 - accuracy: 0.9891 - val_loss: 0.0927 - val_accuracy: 0.9745
Epoch 231/300
15/15 [==============================] - 4s 267ms/step - loss: 0.0356 - accuracy: 0.9893 - val_loss: 0.0998 - val_accuracy: 0.9714
Epoch 232/300
15/15 [==============================] - 4s 267ms/step - loss: 0.0411 - accuracy: 0.9865 - val_loss: 0.0991 - val_accuracy: 0.9722
Epoch 233/300
15/15 [==============================] - 4s 267ms/step - loss: 0.0406 - accuracy: 0.9870 - val_loss: 0.0945 - val_accuracy: 0.9743

In [50]:
with strategy.scope():
    model = VisionTransformer(
        image_size=image_size,
        patch_size=patch_size,
        num_layers=num_layers,
        num_classes=10,
        d_model=d_model,
        num_heads=num_heads,
        mlp_dim=mlp_dim,
        channels=1,
    )
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay),
        metrics=["accuracy"],
    )
    model.load_weights(os.path.join(logdir, "weights", "vit"))
model.evaluate(ds_test)

3/3 [==============================] - 0s 54ms/step - loss: 0.0931 - accuracy: 0.9764


[0.0930701419711113, 0.9764000177383423]

In [51]:
model.evaluate(ds_test)

3/3 [==============================] - 0s 57ms/step - loss: 0.0931 - accuracy: 0.9764


[0.0930701419711113, 0.9764000177383423]